In [1]:
!pip install wandb -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 49.8 MB/s eta 0:00:00


In [1]:
import wandb

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m027 (da24m027-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": [5, 10]},
        "batch_size": {"values": [16, 32, 64]},
        "hidden_layers": {"values": [3, 4, 5]},
        "hidden_size": {"values": [32, 64, 128]},
        "optimizer": {"values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]},
        "activation": {"values": ["sigmoid", "tanh", "ReLU"]},
        "weights": {"values": ["random", "Xavier"]},
        "learning_rate": {"values": [0.001, 0.0001]},
        "weight_decay": {"values": [0, 0.0005, 0.5]},
        "loss_type": {"values": ["mse", "cross_entropy"]},
    },
}

In [4]:
import pprint
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'activation': {'values': ['sigmoid', 'tanh', 'ReLU']},
                'batch_size': {'values': [16, 32, 64]},
                'epochs': {'values': [5, 10]},
                'hidden_layers': {'values': [3, 4, 5]},
                'hidden_size': {'values': [32, 64, 128]},
                'learning_rate': {'values': [0.001, 0.0001]},
                'loss_type': {'values': ['mse', 'cross_entropy']},
                'optimizer': {'values': ['sgd',
                                         'momentum',
                                         'nesterov',
                                         'rmsprop',
                                         'adam',
                                         'nadam']},
                'weight_decay': {'values': [0, 0.0005, 0.5]},
                'weights': {'values': ['random', 'Xavier']}}}


In [5]:
sweep_id = wandb.sweep(sweep_config, project="DA6401_Assignment1")

Create sweep with ID: xy966igu
Sweep URL: https://wandb.ai/da24m027-indian-institute-of-technology-madras/DA6401_Assignment1/sweeps/xy966igu


In [6]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from keras.datasets import fashion_mnist

def preprocess_data(X_train, y_train, X_test, y_test):
    """
    Normalize and reshape dataset.
    """
    X_train, X_test = X_train / 255.0, X_test / 255.0  # Normalize
    X_train, X_test = X_train.reshape(-1, 784), X_test.reshape(-1, 784)  # Flatten images

    # One-hot encoding
    y_train_encoded = np.zeros((y_train.size, 10))
    y_train_encoded[np.arange(y_train.size), y_train] = 1

    y_test_encoded = np.zeros((y_test.size, 10))
    y_test_encoded[np.arange(y_test.size), y_test] = 1

    return X_train, y_train_encoded, X_test, y_test_encoded

def evaluate(model, X_test, y_test):
    """
    Evaluate model accuracy.
    """
    y_pred = model.forward(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_test, axis=1)

    accuracy = np.mean(y_pred_labels == y_true_labels)
    return accuracy

def calculate_loss(y_true, y_pred, loss_type):
  if loss_type == 'mse':
    return np.mean(np.square(y_true - y_pred))
  elif loss_type == 'cross_entropy':
    #Small epsilon to prevent log(0)
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    return -np.mean(y_true * np.log(y_pred))

def calculate_accuracy(y_true, y_pred):
  predicted_classes = np.argmax(y_pred, axis=1)
  true_classes = np.argmax(y_true, axis=1)
  return np.mean(predicted_classes == true_classes)

In [7]:
import numpy as np
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, hidden_size, output_size, activation="ReLU", init_method = "random", loss_type = "mse"):
        self.layers = [input_size] + [hidden_size] * hidden_layers + [output_size]
        self.init_method = init_method
        self.weights = self.initialize_weights()
        self.biases = [np.zeros((1, self.layers[i+1])) for i in range(len(self.layers)-1)]
        self.activation = activation
        self.loss_type = loss_type

    def initialize_weights(self):
        weights = []
        for i in range(len(self.layers) - 1):
            if self.init_method == "Xavier":
                limit = np.sqrt(1 / self.layers[i])
                weights.append(np.random.uniform(-limit, limit, (self.layers[i], self.layers[i+1])))
            else:
                weights.append(np.random.randn(self.layers[i], self.layers[i+1]) * 0.01)
        return weights

    def activation_func(self, x, is_output=False):
        if is_output and self.loss_type == "cross_entropy":
            exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
            return exp_x / np.sum(exp_x, axis=1, keepdims=True)
        if self.activation == "ReLU":
            return np.maximum(0, x)
        elif self.activation == "sigmoid":
            return 1 / (1 + np.exp(-x))
        elif self.activation == "tanh":
            return np.tanh(x)

    def forward(self, X):
        self.a = [X]
        for i, (W, b) in enumerate(zip(self.weights, self.biases)):
            z = np.dot(self.a[-1], W) + b
            # Check if this is the output layer
            is_output = (i == len(self.weights) - 1)
            self.a.append(self.activation_func(z, is_output=is_output))
        return self.a[-1]

In [8]:
import numpy as np

class Backpropagation:
    def __init__(self, model, optimizer="sgd", learning_rate=0.01, momentum=0.9, beta=0.9, beta1=0.9, beta2=0.999, epsilon=1e-8, weight_decay=0.0):
        """
        Initialize the backpropagation optimizer.
        :param model: NeuralNetwork model instance
        :param optimizer: Optimization algorithm ('sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam')
        :param learning_rate: Learning rate for weight updates
        :param momentum: Momentum factor for momentum-based optimizers
        :param beta: Decay factor for RMSprop
        :param beta1: First moment decay rate (Adam, Nadam)
        :param beta2: Second moment decay rate (Adam, Nadam)
        :param epsilon: Small value to prevent division by zero
        :param weight_decay: L2 Regularisation
        """
        self.model = model
        self.optimizer = optimizer
        self.lr = learning_rate
        self.momentum = momentum
        self.beta = beta
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weight_decay = weight_decay

        # Initialize velocity (for Momentum and Nesterov)
        self.velocities = [np.zeros_like(W) for W in model.weights]

        # Initialize moving averages (for Adam, Nadam, RMSprop)
        self.m = [np.zeros_like(W) for W in model.weights]
        self.v = [np.zeros_like(W) for W in model.weights]
        self.t = 0  # Time step for Adam/Nadam

    def backward(self, y_true, y_pred):
        """
        Perform backward pass to compute gradients.
        :param y_true: True labels
        :param y_pred: Predicted output
        :return: Gradients for each layer
        """
        gradients = []
        delta = y_pred - y_true #Same for mse and cross entropy as softmax outputs is handled in feedforward

        for i in reversed(range(len(self.model.weights))):
            grad_W = np.dot(self.model.a[i].T, delta) / y_true.shape[0]
            grad_W += self.weight_decay * self.model.weights[i]  # Apply L2 regularization
            gradients.insert(0, grad_W)

            if i > 0:
                if self.model.activation == "ReLU":
                    delta = np.dot(delta, self.model.weights[i].T) * (self.model.a[i] > 0)
                elif self.model.activation == "sigmoid":
                    delta = np.dot(delta, self.model.weights[i].T) * (self.model.a[i] * (1 - self.model.a[i]))
                elif self.model.activation == "tanh":
                    delta = np.dot(delta, self.model.weights[i].T) * (1 - self.model.a[i]**2)


        return gradients

    def update_weights(self, gradients):
        """
        Updates the model weights using the selected optimizer.
        :param gradients: List of gradient matrices for each layer
        """
        self.t += 1  # Increment time step for Adam/Nadam

        for i in range(len(self.model.weights)):
            grad = gradients[i] + self.weight_decay * self.model.weights[i]  # Apply L2 regularization

            if self.optimizer == "sgd":
                # Stochastic Gradient Descent (SGD)
                self.model.weights[i] -= self.lr * grad

            elif self.optimizer == "momentum":
                # Momentum-Based Gradient Descent
                self.velocities[i] = self.momentum * self.velocities[i] - self.lr * grad
                self.model.weights[i] += self.velocities[i]

            elif self.optimizer == "nesterov":
                # Nesterov Accelerated Gradient Descent
                prev_velocity = self.velocities[i]
                self.velocities[i] = self.momentum * self.velocities[i] - self.lr * grad
                self.model.weights[i] += -self.momentum * prev_velocity + (1 + self.momentum) * self.velocities[i]

            elif self.optimizer == "rmsprop":
                # RMSprop
                self.v[i] = self.beta * self.v[i] + (1 - self.beta) * (grad ** 2)
                self.model.weights[i] -= self.lr * grad / (np.sqrt(self.v[i]) + self.epsilon)

            elif self.optimizer == "adam":
                # Adam (Adaptive Moment Estimation)
                self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad
                self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grad ** 2)

                # Bias correction
                m_hat = self.m[i] / (1 - self.beta1 ** self.t)
                v_hat = self.v[i] / (1 - self.beta2 ** self.t)

                self.model.weights[i] -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

            elif self.optimizer == "nadam":
                # Nadam (Nesterov-accelerated Adam)
                m_hat = (self.beta1 * self.m[i] + (1 - self.beta1) * grad) / (1 - self.beta1 ** self.t)
                v_hat = self.v[i] / (1 - self.beta2 ** self.t)

                self.model.weights[i] -= self.lr * (self.beta1 * m_hat + (1 - self.beta1) * grad / (1 - self.beta1 ** self.t)) / (np.sqrt(v_hat) + self.epsilon)

            else:
                raise ValueError(f"Optimizer '{self.optimizer}' is not recognized.")


In [9]:
def train(config=None):
  with wandb.init(config=config):
    config = wandb.config
    wandb.run.name = "e_{}_hl_{}_hs_{}_lr_{}_opt_{}_bs_{}_init_{}_ac_{}_wd_{}_loss_{}".format(config.epochs,\
                                                                      config.hidden_layers,\
                                                                      config.hidden_size,\
                                                                      config.learning_rate,\
                                                                      config.optimizer,\
                                                                      config.batch_size,\
                                                                      config.weights,\
                                                                      config.activation,\
                                                                      config.weight_decay,\
                                                                      config.loss_type)
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train, y_train, X_test, y_test = preprocess_data(X_train, y_train, X_test, y_test)

    val_size = int(0.1 * len(X_train))
    X_val, y_val = X_train[:val_size], y_train[:val_size]
    X_train, y_train = X_train[val_size:], y_train[val_size:]

    model = NeuralNetwork(input_size=784, hidden_layers=config.hidden_layers, hidden_size=config.hidden_size, output_size=10, activation=config.activation, init_method=config.weights, loss_type = config.loss_type)
    backprop = Backpropagation(model, optimizer=config.optimizer, learning_rate=config.learning_rate, weight_decay=config.weight_decay)

    for epoch in range(config.epochs):
      loss = 0
      for i in range(0, len(X_train), config.batch_size):
        batch_X = X_train[i:i + config.batch_size]
        batch_y = y_train[i:i + config.batch_size]

        # Forward & Backward pass
        y_pred = model.forward(batch_X)
        gradients = backprop.backward(batch_y, y_pred)
        backprop.update_weights(gradients)

        # Compute loss
        loss += calculate_loss(batch_y, y_pred, config.loss_type)

      loss /= len(X_train)
      wandb.log({"epoch": epoch+1, "loss": loss})

      # Validate model
      val_pred = model.forward(X_val)
      val_loss = calculate_loss(y_val, val_pred, config.loss_type)
      val_accuracy = calculate_accuracy(y_val, val_pred)
      wandb.log({"epoch": epoch+1, "val_loss": val_loss, "val_accuracy": val_accuracy})

      print(f"Epoch {epoch+1}/{config.epochs}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

In [10]:
wandb.agent(sweep_id, train, count=100)

wandb: Agent Starting Run: 8glvykow with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	loss_type: mse
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.0005
wandb: 	weights: random


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 2/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 3/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 4/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0965
Epoch 5/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0965
Epoch 6/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 7/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 8/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 9/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963
Epoch 10/10, Loss: 0.0016, Val Loss: 0.1000, Val Accuracy: 0.0963


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
loss,▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁██▁▁▁▁▁
val_loss,▁▁▁███████
epoch,10
loss,0.00156
val_accuracy,0.09633
val_loss,0.1


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1vna9pz3 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	loss_type: cross_entropy
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.5
wandb: 	weights: random


Epoch 1/5, Loss: 0.0072, Val Loss: 0.2303, Val Accuracy: 0.0933
Epoch 2/5, Loss: 0.0072, Val Loss: 0.2303, Val Accuracy: 0.0933
Epoch 3/5, Loss: 0.0072, Val Loss: 0.2303, Val Accuracy: 0.0933
Epoch 4/5, Loss: 0.0072, Val Loss: 0.2303, Val Accuracy: 0.0933
Epoch 5/5, Loss: 0.0072, Val Loss: 0.2303, Val Accuracy: 0.0933


...

Epoch 1/10, Loss: 0.0025, Val Loss: 0.0643, Val Accuracy: 0.5255
Epoch 2/10, Loss: 0.0016, Val Loss: 0.0452, Val Accuracy: 0.6727
Epoch 3/10, Loss: 0.0013, Val Loss: 0.0395, Val Accuracy: 0.7305
Epoch 4/10, Loss: 0.0012, Val Loss: 0.0342, Val Accuracy: 0.7918
Epoch 5/10, Loss: 0.0010, Val Loss: 0.0314, Val Accuracy: 0.8142
Epoch 6/10, Loss: 0.0010, Val Loss: 0.0299, Val Accuracy: 0.8238
Epoch 7/10, Loss: 0.0009, Val Loss: 0.0288, Val Accuracy: 0.8293
Epoch 8/10, Loss: 0.0009, Val Loss: 0.0280, Val Accuracy: 0.8358
Epoch 9/10, Loss: 0.0009, Val Loss: 0.0273, Val Accuracy: 0.8398
Epoch 10/10, Loss: 0.0009, Val Loss: 0.0267, Val Accuracy: 0.8435


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▄▆▇▇█████
val_loss,█▄▃▂▂▂▁▁▁▁
epoch,10
loss,0.00085
val_accuracy,0.8435
val_loss,0.02671


wandb: Agent Starting Run: 4u8k77bo with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	loss_type: mse
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0
wandb: 	weights: Xavier


Epoch 1/10, Loss: 0.0015, Val Loss: 0.0337, Val Accuracy: 0.7943
Epoch 2/10, Loss: 0.0010, Val Loss: 0.0277, Val Accuracy: 0.8340
Epoch 3/10, Loss: 0.0009, Val Loss: 0.0254, Val Accuracy: 0.8510
Epoch 4/10, Loss: 0.0008, Val Loss: 0.0240, Val Accuracy: 0.8595
Epoch 5/10, Loss: 0.0008, Val Loss: 0.0229, Val Accuracy: 0.8655
Epoch 6/10, Loss: 0.0007, Val Loss: 0.0221, Val Accuracy: 0.8677
Epoch 7/10, Loss: 0.0007, Val Loss: 0.0215, Val Accuracy: 0.8705
Epoch 8/10, Loss: 0.0007, Val Loss: 0.0211, Val Accuracy: 0.8733
Epoch 9/10, Loss: 0.0006, Val Loss: 0.0207, Val Accuracy: 0.8767
Epoch 10/10, Loss: 0.0006, Val Loss: 0.0204, Val Accuracy: 0.8785


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▄▆▆▇▇▇███
val_loss,█▅▄▃▂▂▂▁▁▁
epoch,10
loss,0.00063
val_accuracy,0.8785
val_loss,0.02039
